### 사용자 정의 데이터셋
- Pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하(H/W,S/W) 및 시간 소요에 대한 해결책으로 제시
- 대량 데이터셋 전용 처리 모듈 제공
- Dataset과 DataLoader
    * DataSet => 데이터 전처리, 텐서화 등의 작업 진행
    * DataLoader => Dataset 인스턴스를 사용해서 배치크기 만큼 데이터를 추출함

[1] 모듈 로딩 및 데이터 준비
<hr>

In [52]:
# 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder     # 타겟 컬럼 수치화

In [53]:
# 데이터
DATA_FILE=r'C:\VSCode\KDT\Torch_DL\Data\iris.csv'

In [54]:
# CSV => DF
irisDF = pd.read_csv(DATA_FILE)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [55]:
# 타겟 컬럼 수치화 ==> LabelEncodoer
encoder = LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety'] = encoder.transform(irisDF['variety'])

In [56]:
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


[2] 사용자 정의 데이터셋 클래서 생성
<hr>

In [57]:
# -------------------------------------------------------------------
# 클래스 목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스 이름 : CustomDataset
# 부모 클리스 : torch.utils.data.Dataset
# 매개   변수 : featureDF, targetDF
# -------------------------------------------------------------------

In [58]:
class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__ (self,featureDF, targetDF):
        super().__init__()
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows=featureDF.shape[0]
        self.n_featuers=featureDF.shape[1]

    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 데이터 개수 반환 메서드 => Tensor 반환!!!
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS,targetTS
     

In [73]:
# -----------------------------------------------------
# 함수  기능 : 파일 확장자별 데이터프레임 변환 기능
# 함수  이름 : convertDataFrame
# 매개 변수  : data_path
# 함수 결과  : DataFrame
# -----------------------------------------------------
def convertDataFrame(data_path,exit_header=0):
    ext = data_path.rsplit('.')[-1]
    if ext == 'csv':
        return pd.read_csv(data_path, hearder=exit_header)
    elif ext == 'json':
        return pd.read_json(data_path, hearder=exit_header)
    elif ext == ['xlsx','xls']:
        return pd.read_excel(data_path, hearder=exit_header)
    else:
        return pd.read_table(data_path, hearder=exit_header)

In [ ]:
# -----------------------------------------------------
# 클래스기능 : 파일기반 데이터셋
# 클래스이름 : FileDataset
# 매개 변수  : data_path 파일 경로
# 부모 클래스: utils.data.Dataset
# -----------------------------------------------------
class FileCustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__ (self,data_path):
        super().__init__()
        dataDF = convertDataFrame(data_path)
        self.featuresDF=dataDF[dataDF.columns[:-1]]
        self.targetDF=dataDF[dataDF.columns[-1:]]
        
        self.n_features = self.featuresDF.shape[1]
        self.n_rows = self.featuresDF.shape[0]


    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows

    # 데이터 개수 반환 메서드 => Tensor 반환!!!
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS,targetTS

[3] 데이터셋 인스턴스 생성
<hr>

In [59]:
featureDF, targetDF = irisDF[irisDF.columns[:-1]],irisDF[[irisDF.columns[-1]]]
print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')


featureDF => (150, 4), targetDF => (150, 1)


In [60]:
# IRIS 데이터셋 인스턴스 생성
irisDS = CustomDataset(featureDF,targetDF)

In [61]:
# IRIS 데이터셋 속성
irisDS.featureDF, irisDS.n_rows, irisDS.n_featuers

(     sepal.length  sepal.width  petal.length  petal.width
 0             5.1          3.5           1.4          0.2
 1             4.9          3.0           1.4          0.2
 2             4.7          3.2           1.3          0.2
 3             4.6          3.1           1.5          0.2
 4             5.0          3.6           1.4          0.2
 ..            ...          ...           ...          ...
 145           6.7          3.0           5.2          2.3
 146           6.3          2.5           5.0          1.9
 147           6.5          3.0           5.2          2.0
 148           6.2          3.4           5.4          2.3
 149           5.9          3.0           5.1          1.8
 
 [150 rows x 4 columns],
 150,
 4)

In [62]:
# IRIS 데이터셋 메서드
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [68]:
## 필요한 것 : Dataset 인스턴스, Batch_size=5
irisDL=DataLoader(irisDS, batch_size=5)

In [69]:
irisDL.__iter__()

In [71]:
for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)
    break
    

torch.Size([5, 4]) torch.Size([5, 1])
